> # Recognising Handwritten Digits on MNIST DataSet using KNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
mnist =pd.read_csv("../input/digit-recognizer/train.csv")

In [ ]:
mnist
#(rows,cols)

In [ ]:
mnist.columns
#so it represent that first column is label and rest are pixel of label
#in this data image of 28x28 is stored in format of 784 array of pixels

In [ ]:
mnist.head()
#label isalways y
#and we have 784 pixels

In [ ]:
X= mnist.iloc[: ,1:].values #bcz we are accessing integer val # #we can also do like mnist[:,1:]
# accesing number instead of categorical data
y=mnist["label"].values #we can also do like mnist[:,0]
#here we have used .numpy to convert it into numpy array

In [ ]:
X.shape

In [ ]:
y

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
#this just for splitting train and test

In [ ]:

X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
#we can apply kNN on this 

In [ ]:
def plot_image_color(img):
    img=img.reshape((28,28))
    plt.imshow(img) #this function gives u acolor map 
    plt.show()

In [ ]:
plot_image_color(X_train[60])


In [ ]:
def plot_image(img):
    plt.imshow(img.reshape(28,28) ,cmap="gray") #this function gives u acolor map 
    #imshow just plot img and in img u will get (1,784)
    
    #so converting 784 into 28x28  
    plt.show()
    #as the img is graay scale so if we dont put cmap it 
    #automatically put so some color imshow ()
    

In [ ]:
plot_image(X_train[60]) #you can try this out on different image 
print(y_train[60])
#here training timeis o(1) bcz everything is going to happen in prediction time
# and their is no training time

# kNN

In [ ]:
def distance(pA, pB):
    return np.sum((pB - pA)**2)**0.5

In [ ]:
def kNN(X, y, x_query, k = 5):
    """
    X - > (m, 784)  np array (m is just number of eg. images)
    y - > (m,) np array
    x_query -> (1,874) np array
    k -> scaler  int
    
    do knn for classification
    """
    
    m = X.shape[0]
    distances = []
    for i in range(m):
        dis = distance(x_query, X[i])
        distances.append((dis, y[i]))
        
    distances = sorted(distances)
    distances = distances[:k]
    
    distances = np.array(distances)
    labels = distances[:, 1]
    
    
    uniq_label, counts = np.unique(labels, return_counts=True)

    pred = uniq_label[counts.argmax()]
    
    
    return int(pred)

# What algorithm is predicting

In [ ]:
# prediciting label for test
kNN(X_train , y_train, X_test[235], k=7)
#X_test if for testing

# what we are getting image as output

In [ ]:
# displaying test image
plot_image(X_test[235])

In [ ]:
# actual label
y_test[235]
#this for checking that is X_train ==y_train

# accuracy of Knn over test set

# Prediciting 100 Images

In [ ]:
predictions = []
for i in range(100):
    p = kNN(X_train , y_train, X_test[i], k=7)
    predictions.append(p)

In [ ]:
predictions = np.array(predictions)
predictions.dtype

# Accuracy on 100 Images

In [ ]:
p=(y_test[:100] == predictions)
type(p)

In [ ]:
(y_test[:100] == predictions).sum()/len(predictions) #or 100

In [ ]:
predictions

# KNN From sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn=KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
yp=[]
for i in range(100):
    p=knn.predict([X_test[i]])#here we have used it to convert it in 2D array
    p1=np.array(p)
    yp.append(p1)

In [ ]:
yp=np.array(yp)

In [ ]:
yp=yp.reshape((100,)) #this we have done bcz we have earlier converted
#our X_test into 2D array

print(yp.shape)

In [ ]:
(y_test[:100] == yp).sum()/len(yp) #or 100

Applying Another Approach 

In [ ]:
import tensorflow.keras as tf 
mnist_train =pd.read_csv("../input/digit-recognizer/train.csv")
mnist_test =pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
df = pd.DataFrame(mnist_train)#.head(32000)
img= df.drop(["label"], axis=1).values
y= df["label"].values

img = img.reshape(42000, 28, 28, 1)

In [ ]:
img.shape

In [ ]:
model = tf.models.Sequential()
model.add(tf.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.layers.MaxPooling2D((2, 2)))
model.add(tf.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.layers.MaxPooling2D((2, 2)))

model.add(tf.layers.Flatten())
model.add(tf.layers.Dense(64, activation='relu'))
model.add(tf.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


res = model.fit(img, y, epochs=12, validation_split=0.3)


In [ ]:
df_t=pd.DataFrame(mnist_test)
img= df_t.values

img= img.reshape(28000, 28, 28, 1)
prediction = np.argmax(model.predict(img), axis=1)

In [ ]:
# dfinal=pd.DataFrame(data=prediction,columns=["Label"])
#dfinal
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
dfinal= sample_submission.drop(["Label"], axis=1) 
dfinal.insert(1,'Label',prediction)

In [ ]:
dfinal.to_csv("./Recog_MNIST.csv",index=False)